In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

In [25]:
from selenium import webdriver
from selenium_stealth import stealth
import time

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")

options.add_argument("--headless")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
# driver = uc.Chrome(options=options)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

driver.get('https://www.tokopedia.com')
driver.save_screenshot('screenshot.png')

True

In [3]:
# driver = uc.Chrome()
# driver = uc.Chrome(options=chrome_options)
# driver = webdriver.Chrome(options=chrome_options)

In [26]:
df = pd.read_csv('all_clean_scrap.csv')

In [36]:
df = df[df['category'].isin(['aksesoris_pria', 'atasan_pria'])][1:5]

In [39]:
df

,index,Nama,Harga,Link,category,gender
1,1,FRAME KACAMATA VHERKUDARA KHUMA,944000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,aksesoris_pria,pria
2,2,SHERY Nosepad Kacamata Bantalan Penyangga Hidu...,12900,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,aksesoris_pria,pria
3,3,GESPER GASPER BELT IKAT PINGGANG KULIT PRIA AS...,150000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,aksesoris_pria,pria
4,4,Ikat Pinggang Pria Gesper Nylon Canvas Sabuk,110000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,aksesoris_pria,pria


In [40]:
driver.get('https://www.tokopedia.com')

In [41]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [42]:
dct_data = {
    "nama_toko": [],
    "daerah_toko": [],
    "jumlah_terjual": [],
    "stok_tersisa": []
}

In [43]:
def stock_finder(p_stok, stok_total):
    res_stok = p_stok.find_elements(By.CSS_SELECTOR, 'b')[0].text
    res_stok = re.sub(r'\D', '', res_stok)
    if res_stok == '':
        res_stok = 0
    else:
        res_stok = int(res_stok)
    stok_total += res_stok

    return stok_total

In [44]:
index_stopped=0
counter = 0
for link in df['Link']:
    print('counter now', counter)
    if counter < index_stopped:
        counter += 1
        continue
    else:
        counter += 1

    # EARLY STOP
    if counter == 10:
        break

    #Try if the web is still not ready
    try_time = 5
    for i in range(try_time):
        try:    
            driver.get(link)
            time.sleep(2)

            # Nama Toko data
            h2_nama_toko = driver.find_element(By.XPATH, '//h2[@class="css-1wdzqxj-unf-heading e1qvo2ff2"]')
            break
        except:
            try_time -= 1
    if try_time == 0:
        print(f'Error loading page after 5 try time')
        continue
    
    dct_data['nama_toko'].append(h2_nama_toko.text)


    # Daerah Data
    try:
        h2_daerah = driver.find_element(By.XPATH, '//h2[@class="css-1pd07ge-unf-heading e1qvo2ff2"]')
        nama_daerah = h2_daerah.find_elements(By.CSS_SELECTOR, 'b')[0].text
    except:
        nama_daerah = 'Unknown/Dilayani Tokopedia'

    dct_data['daerah_toko'].append(nama_daerah)

    # Terjual Data
    try:
        p_sold = driver.find_element(By.XPATH, '//p[@data-testid="lblPDPDetailProductSoldCounter"]')
        sold_val = int(re.sub(r'\D', '', p_sold.text)) 
        if 'rb' in p_sold.text:
            sold_val = sold_val * 1000
    except:
        sold_val = 0

    dct_data['jumlah_terjual'].append(sold_val)
    
    # Stok Data
    stok_total = 0
    try:
        div_variant = driver.find_element(By.XPATH, '//div[@class="css-1b2d3hk"]')
        parent_level_div = div_variant.find_elements(By.CSS_SELECTOR, 'div')
        all_level_div = [x for x in parent_level_div if x.get_attribute('class')=='css-hayuji']

        len_level1 = len(all_level_div[0].find_elements(By.CSS_SELECTOR, 'button'))

        stok_total = 0
        for el_level1 in all_level_div[0].find_elements(By.CSS_SELECTOR, 'button'):
            el_level1.click()

            if len(all_level_div) >= 2:
                for el_level2 in all_level_div[1].find_elements(By.CSS_SELECTOR, 'button'):
                    el_level2.click()
                    p_stok = driver.find_element(By.XPATH, '//p[@class="css-1yy88m3-unf-heading e1qvo2ff8"]')
                    stok_total = stock_finder(p_stok, stok_total)

            else:
                p_stok = driver.find_element(By.XPATH, '//p[@class="css-1yy88m3-unf-heading e1qvo2ff8"]')
                stok_total = stock_finder(p_stok, stok_total)
    except:
        p_stok = driver.find_element(By.XPATH, '//p[@class="css-1yy88m3-unf-heading e1qvo2ff8"]')
        stok_total = stock_finder(p_stok, stok_total)
    

    dct_data['stok_tersisa'].append(stok_total)
    

counter now 0
counter now 1
counter now 2
counter now 3


In [45]:
index_stopped = counter

In [46]:
dct_data

{'nama_toko': ['Vherkudara Official',
  'Shery Official Shop',
  'NESZ_leather',
  'AndreasLukitaShop'],
 'daerah_toko': ['Kab. Sleman',
  'Jakarta Pusat',
  'Jakarta Utara',
  'Jakarta Utara'],
 'jumlah_terjual': [30, 1000, 4000, 5000],
 'stok_tersisa': [8, 3207, 104, 1211]}

In [48]:
df['stok_tersisa'] = dct_data['stok_tersisa']
df['daerah_toko'] = dct_data['daerah_toko']
df['nama_toko'] = dct_data['nama_toko']

In [56]:
from datetime import datetime, timedelta

# Get the current date and time
now = datetime.now()

# Subtract one year (approximate by subtracting 365 days)
one_year_ago = now - timedelta(days=731)

# Format the date as a string in the desired format
date_string = one_year_ago.strftime("%Y-%m-%d")

print(date_string)

2022-07-31


In [57]:
df['created_at'] = date_string

In [58]:
df

,index,Nama,Harga,Link,category,gender,stok_tersisa,daerah_toko,nama_toko,created_at
1,1,FRAME KACAMATA VHERKUDARA KHUMA,944000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,aksesoris_pria,pria,8,Kab. Sleman,Vherkudara Official,2022-07-31
2,2,SHERY Nosepad Kacamata Bantalan Penyangga Hidu...,12900,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,aksesoris_pria,pria,3207,Jakarta Pusat,Shery Official Shop,2022-07-31
3,3,GESPER GASPER BELT IKAT PINGGANG KULIT PRIA AS...,150000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,aksesoris_pria,pria,104,Jakarta Utara,NESZ_leather,2022-07-31
4,4,Ikat Pinggang Pria Gesper Nylon Canvas Sabuk,110000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,aksesoris_pria,pria,1211,Jakarta Utara,AndreasLukitaShop,2022-07-31


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 1 to 4
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         4 non-null      int64 
 1   Nama          4 non-null      object
 2   Harga         4 non-null      int64 
 3   Link          4 non-null      object
 4   category      4 non-null      object
 5   gender        4 non-null      object
 6   stok_tersisa  4 non-null      int64 
 7   daerah_toko   4 non-null      object
 8   nama_toko     4 non-null      object
 9   created_at    4 non-null      object
dtypes: int64(3), object(7)
memory usage: 524.0+ bytes


In [61]:
from google.cloud import storage
import os
import tempfile

# Set up Google Cloud Storage client
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "datahub-project-430801-c216773b2d70.json"
client = storage.Client()

In [63]:
# Use tempfile to create a temporary file
with tempfile.NamedTemporaryFile(suffix='.csv') as tmp:
    local_file_path = tmp.name
    df.to_csv(local_file_path, index=False)

    # Define the GCS bucket and blob (file) name
    bucket_name = 'data_scrap_azhar'
    destination_blob_name = f'result_data/atasan_pria_{date_string}.csv'

    # Upload the file to GCS
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(local_file_path)

    print(f'File {local_file_path} uploaded to {destination_blob_name}.')


File /var/folders/dj/snpp_k5j5tn5sz1sjr3_bh6h0000gn/T/tmp6lq7w6jn.csv uploaded to result_data/atasan_pria_2022-07-31.csv.
